# Merge methods (mutating joins)

![alt text](https://storage.googleapis.com/campus-cvs/lectures/3.4%20Merge%20Methods.png "Merge Methods")

**IMPORTANTE:** En esta clase se asume que los datos ya se encuentran en un formato comúnmente conocido como “tidy data”, es decir, donde cada fila corresponde a una única observación y cada columna contiene valores correspondientes a una única variable. En esta lectura, utilizaremos de ejemplo un data set que ya cumple con la anterior definición.

### Introducción

En la práctica, es raro que un análisis de datos involucre solamente una tabla. Normalmente tendrás muchas tablas contribuyendo a tu análisis y necesitas herramientas flexibles para combinarlas.

Si ha utilizado algún lenguaje de consulta a una base de datos, como SQL, recordará que hay ciertas operaciones que permiten unificar dos o más de esas tablas, complementando los valores entre ellas. A este tipo de operación se le llama **JOIN**

Hay 3 operaciones fundamentales que se pueden realizar con un **JOIN**:

- **Uniones uno a uno:** por ejemplo, al unir dos tablas por su columna de índices o ID (que debe contener valores únicos).

- **Uniones de muchos a uno:** por ejemplo, cuando se une un índice (único) a una o más columnas en una tabla diferente.

- **Uniones de muchos a muchos:** unión donde el criterio de índices abarca dos o más columnas. Por ejemplo, cuando queremos unir una tabla de aviones con una de vuelos y necesitamos unir mediante la aerolínea y el ID de vuelo, pues si solo lo hacemos mediante una de ellas, podría haber duplicados o no haber coincidencias.

#### Nota:
Al unir columnas en columnas (potencialmente una combinación de muchos a muchos), se descartarán los índices de las tablas pasadas.

Vale la pena dedicar un tiempo a comprender el resultado del caso de unión de muchos a muchos. En SQL / álgebra relacional estándar, si una combinación de índices aparece más de una vez en ambas tablas, la tabla resultante tendrá el producto cartesiano de los datos asociados. Aquí hay un ejemplo muy básico con una combinación de teclas única:

In [1]:
import pandas as pd

In [6]:
left = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)
left

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [7]:
right = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)
right

,key,C,D
0,K0,C0,D0
1,K1,C1,D1
2,K2,C2,D2
3,K3,C3,D3


In [8]:
pd.merge(left, right, on="key")

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


Esta es otra forma de hacerlo, invocando a `merge` como un método de la clase DataFrame. El dataframe sobre el que se aplica el método, se considera el dataframe de la izquierda

In [11]:
left.merge(right, on="key")

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


### Merge sobre múltiples columnas

A continuación, se muestra un ejemplo más complicado donde hay dos columnas que se utilizarán columnas de ID (`key1`,`key2`). Solo están presentes las claves que aparecen a la izquierda y a la derecha (la intersección), ya que tenemos el argumento `how='inner'` por defecto. 

In [15]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

In [16]:
right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

In [17]:
pd.merge(left, right, on=["key1", "key2"])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


## Merge methods

El argumento `how` del método `merge` especifica cómo determinar qué variables (columnas) serán incluidas en la tabla resultante. Si una valores no aparece en las tablas de la izquierda o de la derecha, los valores de la tabla combinada serán `NA`. Aquí hay un resumen de las opciones de `how` y sus nombres equivalentes de SQL:

| Merge method | Nombre del SQL Join | Descripción                                  |
| ------------ | ------------------- | -------------------------------------------- |
| left         | LEFT OUTER JOIN     | Usa únicamente las variables de la izquierda |
| right        | RIGHT OUTER JOIN    | Usa únicamente las variables de la derecha   |
| outer        | FULL OUTER JOIN     | Utiliza la unión de las variable de ambos data frames |
| inner        | INNER JOIN          | Utiliza la intersección de las variables de ambos data frames |

### Left Join

In [18]:
pd.merge(left, right, how="left", on=["key1", "key2"])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


### Right Join

In [19]:
pd.merge(left, right, how="right", on=["key1", "key2"])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


### Outer Join

In [20]:
pd.merge(left, right, how="outer", on=["key1", "key2"])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


### Inner Join

In [21]:
pd.merge(left, right, how="inner", on=["key1", "key2"])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


#### Nota:

Esos fueron los tipos de join que puedes realizar entre data frames.

El método `merge` acepta más argumentos en función de los casos especiales que se te vayan presentando. Para esos casos, te recomiendo consultar la [documentación](https://pandas.pydata.org/docs/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging).